<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<br />
<br />

<a target="_blank" href="https://colab.research.google.com/github/wandb/weave/blob/master/examples/prompts/llm_monitoring/openai_proxy_quickstart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Note:** you will need an [OpenAI API key](https://platform.openai.com/account/api-keys) to run this colab.

# Openai Proxy Quickstart

A simple tool to capture metadata about your interactions with the OpenAI API.  Currently supports ChatCompletion endpoint, including streaming.

# Step 0: Setup
Install dependencies, authenticate with OpenAI and W&B via API keys.

In [ ]:
# Note: `weave` and `tiktoken` are not required for the proxy,
# but are used in the last cell example

!pip install -qqq openai weave tiktoken

In [ ]:
# authenticate with OpenAI
from getpass import getpass
import os

if os.getenv("OPENAI_API_KEY") is None:
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

In [ ]:
# authenticate with W&B
from getpass import getpass
import os

if os.getenv("WANDB_API_KEY") is None:
  os.environ["WANDB_API_KEY"] = getpass("Paste your W&B API key from: https://wandb.ai/authorize\n")
print("W&B API key configured")

# Step 1: Update OpenAI API key and base url

All you need to do is:
1. Change your OpenAI API key to the format `{wandb_api_key}:{openai_api_key}`; and
2. Change your OpenAI API base url to `https://proxy.wandb.ai/proxy/openai/v1`

In [ ]:
import os
import openai

openai_api_key = os.getenv("OPENAI_API_KEY")
wandb_api_key = os.getenv("WANDB_API_KEY")

In [ ]:
openai.api_base = "https://proxy.wandb.ai/proxy/openai/v1"
openai.api_key = f"{wandb_api_key}:{openai_api_key}"

# Step 2: Start logging!

You can use your existing API calls as-is, now with extra logging!

By default, these logs will be saved in a StreamTable named `{your_entity}/monitoring/openai`, where the project is `monitoring` and the table name is `openai`. `your_entity` will match the default entity associated with your [W&B API key](wandb.ai/authorize). You can optionally set this field to any different wandb username or team name to which you have login access.
# Visualize your logs in an interactive Weave Board

**View your logs at [https://weave.wandb.ai/browse/wandb](https://weave.wandb.ai/browse/wandb)/{your_entity}/monitoring/table/openai**.

Click on "+ New board from template" on the right hand side to open an interactive Weave Board. This Board will automatically monitor OpenAI API usage and visualize key metrics like cost and latency across all your calls. You can also extend this board with custom analysis, filter/sort/group by attributes of interest, and share your visualizations and insights with your team.

# Examples of logging methods

There are a few different ways to call OpenAI with the proxy shown below:

## Example 1: Using OpenAI SDK

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Tell me a joke about gradients!"}],
)
response

## Example 2: Using CURL

In [ ]:
%%bash
curl "https://proxy.wandb.ai/proxy/openai/v1/chat/completions" \
-H "Authorization: Bearer $WANDB_API_KEY:$OPENAI_API_KEY" \
-H "Content-Type: application/json" \
-d '{
     "model": "gpt-3.5-turbo",
     "messages": [{"role": "user", "content": "Tell me a joke about loss functions!"}]
   }'

# Optional configuration
Optionally specify headers to determine where you want your data to be created.

| Header | Description | Default |
| --- | --- | --- |
| `X-Wandb-Entity` | W&B entity name | Your default entity |
| `X-Wandb-Project` | W&B project name | `monitoring` |
| `X-Wandb-Stream` | StreamTable name | `openai` |

By default, proxy data will show up in `{your_entity}/monitoring/openai`.

### Advanced configuration
1. Optionally add custom attributes to your table with the `X-Wandb-Attribute-` prefix.  For example, adding the header `X-Wandb-Attribute-Foo: bar` will add a column `Foo` with the value `bar` to your table.
2. Optionally add the `X-Wandb-Client-Id` header if you want to further group related requests together.  By default, each request will have its own unique Client ID.

In [ ]:
WB_ENTITY = # set to your wandb username or team name
WB_PROJECT = "weave" # top-level directory for this work
STREAM_NAME = "openai_logs" # record table which stores the logs of OpenAI API calls as they stream in

## Example 3: Using OpenAI SDK

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Tell me a joke about gradients as if you were Shakespeare"}],
    headers={
        "X-Wandb-Entity": WB_ENTITY,
        "X-Wandb-Project": WB_PROJECT,
        "X-Wandb-Stream": STREAM_NAME,
        "X-Wandb-Client-Id": "Shakespearean-Gradients-001",
        "X-Wandb-Attribute-Persona": "Shakespeare",
    }
)
response

## Example 4: Using CURL

**Note:** Replace WB_ENTITY, WB_PROJECT, and STREAM_NAME with explicit strings below to specify where to store the logs.
You will likely get an authentication error if you don't do this.

In [ ]:
%%bash
curl "https://proxy.wandb.ai/proxy/openai/v1/chat/completions" \
-H "Authorization: Bearer $WANDB_API_KEY:$OPENAI_API_KEY" \
-H "Content-Type: application/json" \
-H "X-Wandb-Entity: WB_ENTITY" \
-H "X-Wandb-Project: WB_PROJECT" \
-H "X-Wandb-Stream: STREAM_NAME" \
-H "X-Wandb-Client-Id: Shakespearean-Gradients-001" \
-H "X-Wandb-Attribute-Persona: Shakespeare" \
-d '{
     "model": "gpt-3.5-turbo",
     "messages": [{"role": "user", "content": "Tell me a joke about loss functions!"}]
   }'